In [17]:
from process import IndexMapping

ImportError: cannot import name 'IndexMapping' from 'process' (c:\Users\s381731\OneDrive - Cranfield University\Documents\Python\Elicie\atten_bigru_seq2seq\process.py)

In [ ]:
with open('intermedium/processed_movie_lines.txt', 'r') as f:
    read_in = f.read().strip().split('\n')

In [ ]:
pairs = [[s for s in l.split('\t')] for l in read_in]

In [ ]:
pairs[:5]

[['not the hacking and gagging and spitting part. please.',
  'okay... then how bout we try out some french cuisine. saturday? night?'],
 ['you re asking me out. that s so cute. what s your name again?',
  'forget it.'],
 ['no no it s my fault we didn t have a proper introduction', 'cameron.'],
 ['gosh if only we could find kat a boyfriend...',
  'let me see what i can do.'],
 ['that s because it s such a nice one.', 'forget french.']]

In [ ]:
index_mapping = NumericalMapping()

In [ ]:
for p in pairs:
    index_mapping.add_sentence(p[0])
    index_mapping.add_sentence(p[1])

In [ ]:
index_mapping.n_words

28409

In [ ]:
index_mapping.word2index

{'not': 2,
 'the': 3,
 'hacking': 4,
 'and': 5,
 'gagging': 6,
 'spitting': 7,
 'part.': 8,
 'please.': 9,
 'okay...': 10,
 'then': 11,
 'how': 12,
 'bout': 13,
 'we': 14,
 'try': 15,
 'out': 16,
 'some': 17,
 'french': 18,
 'cuisine.': 19,
 'saturday?': 20,
 'night?': 21,
 'you': 22,
 're': 23,
 'asking': 24,
 'me': 25,
 'out.': 26,
 'that': 27,
 's': 28,
 'so': 29,
 'cute.': 30,
 'what': 31,
 'your': 32,
 'name': 33,
 'again?': 34,
 'forget': 35,
 'it.': 36,
 'no': 37,
 'it': 38,
 'my': 39,
 'fault': 40,
 'didn': 41,
 't': 42,
 'have': 43,
 'a': 44,
 'proper': 45,
 'introduction': 46,
 'cameron.': 47,
 'gosh': 48,
 'if': 49,
 'only': 50,
 'could': 51,
 'find': 52,
 'kat': 53,
 'boyfriend...': 54,
 'let': 55,
 'see': 56,
 'i': 57,
 'can': 58,
 'do.': 59,
 'because': 60,
 'such': 61,
 'nice': 62,
 'one.': 63,
 'french.': 64,
 'there.': 65,
 'where?': 66,
 'word.': 67,
 'as': 68,
 'gentleman': 69,
 'sweet.': 70,
 'sure': 71,
 'have.': 72,
 'really': 73,
 'wanna': 74,
 'go': 75,
 'but': 

In [ ]:
PAD = 0
SOS = 1
EOS = 2

In [ ]:
def word_indexing(mapping, sent):
    """
    Represent each word in the given sentence by an integer defined in the created indexing mapping

    Args:
        mapping (dict): the established mapping between words and integers
        sent (str): The sentence for indexing
    Return:
        (list<int>): A list of integers for word representation
    """
    return [mapping.word2index[word] for word in sent.split(' ')] + [EOS]

In [ ]:
sample_sent = pairs[0][0]
print(sample_sent)
print(word_indexing(index_mapping, sample_sent))

not the hacking and gagging and spitting part. please.
[2, 3, 4, 5, 6, 5, 7, 8, 9, 2]
